In [17]:
import pandas as pd                               # panda
import pandas_datareader as pdr                   # Baixar dados yahoo

import matplotlib.pyplot as plt                   # grafico
import matplotlib.dates as mdates                 # trabalhar datas

import statsmodels.api as sm                      # regressao
from statsmodels.api import add_constant          # para a regressao considerar o intercepto

import numpy as np
import scipy.stats
#import seaborn as sns

from statsmodels.tsa.stattools import coint       # Funcao Coint ()
from statsmodels.tsa.stattools import adfuller

# Declaracoes
pd.options.display.float_format = '{:.2f}'.format

#!pip install MetaTrader5
#!pip install --upgrade MetaTrader5
import MetaTrader5 as mt5
from datetime import datetime
from datetime import date
import time
import pytz

import ipywidgets

%matplotlib inline

### DECLARACAO DE FUNCOES ###

In [18]:
# Baixa base de dados e retorna uma base da forma [data,close_ativo_1, close_ativo_2]
def baixar_yahoo(tickers,inicio,fim,baixayahoo):

    # Percorre a lista tickers, lendo o yahoo e salvando no DataFrame db
    db = pd.DataFrame()
    if (baixayahoo == True):
        
        for i in tickers:
            cot = pdr.DataReader(i, 'yahoo', inicio,fim)
            cot['Ativo']=i
            db = pd.concat([db,cot],sort=True)  

        db.to_csv('Ativos.csv')
        #db.to_excel('ativos.xlsx')
    
    db = pd.read_csv('Ativos.csv',parse_dates=[0], infer_datetime_format = True)
    #db = pd.read_excel('ITSA4GOLL4.xlsx')
    db = db[['Ativo','Date','Open','High','Low','Adj Close']].rename(columns={'Adj Close': 'Close'})    
    
    # Cria Tabela com os fechamentos dos dois ativos [DATA, Close1, Close2]
    esquerda = db[db['Ativo']==tickers[0]].set_index('Date')['Close']
    direita = db[db['Ativo']==tickers[1]].set_index('Date')['Close']
    par = pd.merge(left=esquerda, right=direita, left_on='Date', right_on='Date').rename(columns={'Close_x': tickers[0], 'Close_y': tickers[1]})

    #par.to_excel('Par.xlsx')
    
    return par

def Get_data(tickers,inicio,fim,janela):

    # Checa se ja estamos conectados, se não, conecta
    if (mt5.terminal_info()==None):
          
        # conecte-se ao MetaTrader 5
        if not mt5.initialize():
            print("initialize() failed, error code =",mt5.last_error())
            mt5.shutdown()
        
    #quebra as strings dates(inicio,fim) recebidas e transformando em integer
    inicio_= inicio.split('-')
    inicio = [int(element) for element in inicio_]
    fim_= fim.split('-')
    fim = [int(element) for element in fim_]

    timezone = pytz.timezone("Etc/UTC")
    
    #Verifica caso seja timeframe diario ou intraday
    if janela == '1d':
        dt_inicio = datetime(inicio[0], inicio[1], inicio[2], tzinfo=timezone)
        dt_fim = datetime(fim[0], fim[1], fim[2], tzinfo=timezone)
        timeframe = mt5.TIMEFRAME_D1
    #aqui trata as datas intraday
    else:
        dt_inicio = datetime(inicio[0], inicio[1], inicio[2], inicio[3], inicio[4], tzinfo=timezone)
        dt_fim = datetime(fim[0], fim[1], fim[2], fim[3], fim[4], tzinfo=timezone)
        timeframe = mt5.TIMEFRAME_M5
        
    # obtendo o par de acoes com base nas datas enviadas nas strings, caso len seja 1, getdata retorna o dataframe de uma unica acao
    if (len (tickers) == 1):        
        stock1 = mt5.copy_rates_range(tickers[0], timeframe, dt_inicio, dt_fim)    
        stock_df = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
                
    else:    
        stock1 = mt5.copy_rates_range(tickers[0], timeframe, dt_inicio, dt_fim)
        stock2 = mt5.copy_rates_range(tickers[1], timeframe, dt_inicio, dt_fim)
        esquerda = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
        direita = pd.DataFrame(stock2)[['close','time']].rename(columns={'close': tickers[1]})
        stock_df = pd.merge(left=esquerda, right=direita, left_on='time', right_on='time')
    
    #Se o timeframe for intraday, retorna um datetime (com minutos e horas), caso contrario, retorna um date
    if (janela != '1d'):
        stock_df['Date'] = stock_df['time'].apply(lambda x: str (datetime.fromtimestamp(x, tz=timezone))[:-9]) #define timezone
    else:
        stock_df['Date'] = stock_df['time'].apply(lambda x: date.fromtimestamp(x+21600)) #adicionando 21600 pra corrigir o fuso no momento de converter tsp em str
        stock_df.drop('time',axis=1,inplace = True)
        
    return stock_df.set_index('Date')

In [19]:
def regressao_univariada(x, y):
    
    # Gera a REGRESSAO LINEAR UNIVARIADA -> y = b*x + c + e
    X = sm.add_constant(x)
    res = sm.OLS(y,X).fit()
    coef_ang=res.params[1]
    residuo = par_subset[tickers[0]]-res.predict()
    zscore  = (residuo - np.mean(residuo))/np.std(residuo);
    return coef_ang, residuo, zscore 

def regressao_multivariada(x, y, periodo):  
    
    # Gera a REGRESSAO LINEAR MULTIVARIADA (QuantGo "Simples") -> y = b*x + c*t + d + e
    X = np.column_stack((x, range(1,periodo+1,1)))
    X = sm.add_constant(X)
    res = sm.OLS(y,X).fit()
    coef_ang=res.params[1]
    residuo = y-res.predict()
    zscore  = (residuo - np.mean(residuo))/np.std(residuo);
    return coef_ang, residuo, zscore, res


def regressao_residuos(residuos):
  
    residuos_shifted = residuos.diff(1).fillna(method="bfill")  
    delta = residuos - residuos_shifted
    print(residuos_shifted)
    X = sm.add_constant(residuos_shifted, prepend=True)
    res = sm.OLS(residuos,X).fit()
    print(res.summary)
    return res.params[1]

# ADF da Statsmodel
def adftest(df,reg):

    adf  = adfuller(df,maxlag=1, autolag="BIC")
    
    ## %ADF clássico, de acordo com o output da função
    if   adf[0] < adf[4]['1%']:  adfc='99%'
    elif adf[0] < adf[4]['5%']:  adfc='95%'
    elif adf[0] < adf[4]['10%']: adfc='90%'
    else:                        adfc='-'                 

    ## %ADF do Ferro
    # Utiliza duas tabelas - uma para quando o tempo é significante, outra quando não é (indep. do noobs)
    ttest = reg.params[1]/reg.bse[1]
    critical_value_tempo = scipy.stats.t.ppf(0.01,df=(len(df)-3)) # Retorna o inverso bicaudal da distribuição t de Student

    if   adf[0] < -4.32:         adfc_2='99%'
    elif adf[0] < -3.67:         adfc_2='95%'
    elif adf[0] < -3.28:         adfc_2='90%'
    else:                        adfc_2='-' 
        
    if   adf[0] < -3.58:         adfc_3='99%'
    elif adf[0] < -3.22:         adfc_3='95%'
    elif adf[0] < -2.60:         adfc_3='90%'
    else:                        adfc_3='-'                 

    
    if (abs(ttest) < critical_value_tempo): aceitar_t0 = '99%'
    else:  aceitar_t0 = '0%'
        
        
    return adf[0], adfc, adf[1], adfc_2, adfc_3, aceitar_t0      

def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''
    
    r, p = scipy.stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = scipy.stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    
    return r, p, lo, hi
    
def pct_financeiro(x,y,coef_ang,residuo):
    
    ultimo_x = x.tail(1)[0]
    ultimo_y = y.tail(1)[0]
    
    fin_x = ultimo_x*coef_ang
    fin_y = ultimo_y
    
    if (residuo > 0):
        compra = fin_x
        venda = fin_y
    else:
        compra = fin_y
        venda = fin_x
 
    cv = "{:.0%}".format(compra/venda)
    return cv

def calculo_meia_vida(residuo):
    # Retirado da planilha do Ferro
    coef_ang_residuos = regressao_residuos(residuo) 
    mv_beta = -1*np.log(1+coef_ang_residuos)
    meia_vida = 2/mv_beta  
    return meia_vida

def calculo_meia_vida1(residuo):
    # Retirado de um paper e adaptado pela formula da planilha do Ferro
    price = pd.Series(residuo)  
    lagged_price = price.shift(1).fillna(method="bfill")  
    delta = price - lagged_price  
    beta = np.polyfit(lagged_price, delta, 1)[0] 
    #half_life = ((-2*np.log(2))/beta)  # paper (varios), mas sem o 2*
    half_life = 2/(-1*np.log(1+beta))   # planilha ferro
     
    return "{} Dias".format(int(round(half_life)))

In [20]:
## Funcoes graficas
def grafico_residuo(residuo):
        
    plt.figure(figsize=(20,4))
    ax = residuo.plot(color='g', grid=True, label='Z-Score')
    plt.axhline(residuo.mean(), color='red')
    plt.axhline(residuo.mean()+2*residuo.std(), color='blue')
    plt.axhline(residuo.mean()-2*residuo.std(), color='blue')

    ax.xaxis_date()  # formata o timestamp para o formato data
    ax.set_axisbelow(True)
    ax.set_title('Residuos', color='black')
    ax.set_facecolor('white')
    ax.figure.set_facecolor('white')

    plt.legend()
    plt.show()    

In [55]:
def ajusta_B3Date (data_start, qtde_dias):
    
    df = pd.read_excel('Calendar.xlsx')
    df['Date'] = df['Date'].apply(str)
    index = df [df['Date'] == data_start].index.values.astype(int)[0]
    data_fim = df.iloc[index+qtde_dias][1]
    
    return (data_fim + '-17-00')

    
def simula_trade (ticker, beta_inicio, dp_entrada, dp_target, start_time , hlife):
    
    #passando a data de inicio do trade para identificarmos qual a data limite para o trade com base no calendario da B3
    end_time = ajusta_B3Date(start_time[:-6],hlife)
    #print (start_time)
    #print (end_time)
    stock = Get_data(ticker, start_time, end_time,'M5')
    
    return stock
    
def merge_intraday (ticker, dt_inicio, dt_intraday):

    dt_temp = dt_intraday.split('-')
    dt_temp = [int(element) for element in dt_temp]
    dt_temp[2] = dt_temp[2] - 1
    dt_fim = str(dt_temp[0])+'-0'+str(dt_temp[1])+'-'+str(dt_temp[2])
    stock  = Get_data(ticker, dt_inicio,dt_fim,'1d')
    stock_intraday = Get_data(ticker, dt_intraday,dt_intraday,'M5')
    stock_intraday['Date'] =  stock_intraday['time'].apply(date.fromtimestamp)
    stock_intraday.set_index('Date', inplace = True)
    stock_concat = pd.concat([stock,stock_intraday])
    stock_concat.drop('time',axis=1,inplace = True)

    return stock_concat

In [56]:
def Get_Cubo (tickers, dt_cubo, intra_bt):
    
    #Pegando a data B3 de 250 regoes atras
    if intra_bt:
        dt_inicio = ajusta_B3Date(dt_cubo[:-6],-250)
    else:
        dt_inicio = ajusta_B3Date(dt_cubo,-250)
    
    print (dt_inicio)
    
    if intra_bt:
        #Para retornar o par com dados diarios, com ultimo dia sendo o intraday passado em dt_final
        par = merge_intraday (tickers, dt_inicio, dt_cubo)
    else:   
        # Busca cotacoes no Yahoo Finance
        par = Get_data(tickers, dt_inicio, dt_cubo,'1d')

    par['ratio']  = par[tickers[0]]/par[tickers[1]]
    par['ratio1'] = par[tickers[1]]/par[tickers[0]]

    # Cria o cubo de periodos
    cubo = pd.DataFrame(columns=['Periodo','Dickey_Fuller','ADF','ADF_0pct','TEMPO','ADF_99pct','Coef_Ang','QTD_Desvios','Fisher_min','Fisher_max'
                                 ,'ADF_pvalue','pct_fin','meia_vida'
                                 ,#'coint_p', 'coint_dickey','coint_adf'
                                ])

    # Popula o cubo de periodos
    #for i in range(100,260,10):
    for i in [100,120,140,160,180,200,220,240,250]:

        # Cria subset do tamanho do periodo atual da iteração e seta a variavel indep (x) e dependente (y)
        par_subset = par.tail(i)
        x = par_subset[tickers[1]]
        y = par_subset[tickers[0]]

        # Adiciona o PERIODO analisado no dataframe
        cubo = cubo.append({'Periodo': int(i)}, ignore_index=True)

        # Gera a REGRESSAO e retorna o Coef Ang, os residuos e o z-score. 
        # Retorna tbm o vetor de retorno da regressao para ser usado no ADF
        coef_ang, residuo, zscore, reg  = regressao_multivariada(x, y, i)

        # Teste de Estacionariedade dos Resíduos (ADF)
        adfstat,adfc,adfpvalue,adfc_2,adfc_3,aceitar_t0 = adftest(residuo, reg)

        # Calculo do Fisher
        ficher_r, fisher_pvalue, fisher_lo, fisher_hi = pearsonr_ci(x.diff().fillna(method="bfill"), y.diff().fillna(method="bfill"))

        # Calculo do % Financeiro (C/V)
        pct_fin = pct_financeiro(x,y,coef_ang, zscore.tail(1).values)

        # Calculo da Meia-vida - ORNSTEIN-UHLENBECK
        meia_vida = calculo_meia_vida1(residuo)


        # Preenchimento do cubo
        cubo.loc[cubo['Periodo']==i, 'Coef_Ang'     ] = coef_ang 
        cubo.loc[cubo['Periodo']==i, 'QTD_Desvios'  ] = zscore.tail(1)[0]
        cubo.loc[cubo['Periodo']==i, 'Dickey_Fuller'] = adfstat
        cubo.loc[cubo['Periodo']==i, 'ADF'          ] = adfc 
        cubo.loc[cubo['Periodo']==i, 'ADF_pvalue'   ] = adfpvalue 
        cubo.loc[cubo['Periodo']==i, 'Fisher_min'   ] = "{:.0%}".format(fisher_lo)
        cubo.loc[cubo['Periodo']==i, 'Fisher_max'   ] = "{:.0%}".format(fisher_hi)
        cubo.loc[cubo['Periodo']==i, 'pct_fin'      ] = pct_fin
        cubo.loc[cubo['Periodo']==i, 'meia_vida'    ] = meia_vida
        cubo.loc[cubo['Periodo']==i, 'ADF_0pct'     ] = adfc_2
        cubo.loc[cubo['Periodo']==i, 'ADF_99pct'    ] = adfc_3
        cubo.loc[cubo['Periodo']==i, 'TEMPO'        ] = aceitar_t0

    #par.to_excel("Par.xlsx")    
    display(cubo)

    # Desvio médio do Coef. Angular - apenas dos periodos cointegrados
    cubo_cointegrado = cubo.loc[cubo['ADF_0pct'] != '-']
    print("Desvio Medio do Coef. Angular: {:.2%}" .format(cubo_cointegrado['Coef_Ang'].std()/cubo_cointegrado['Coef_Ang'].mean()))
    beta_vol = cubo_cointegrado['Coef_Ang'].std()/cubo_cointegrado['Coef_Ang'].mean()
    
    return cubo, beta_vol

Atencao

- Calculo da meia vida: o calculo que peguei da planilha difere com os que encontrei na internet. Nenhum deles batia, fiz um mix dos dois via tentativa e erro e agora está bem proximo.. mas acho que temos que entender melhor

- Fisher: baixei uma formula, nao entendi nada.. tbm seria legal entender

- ADF: estou testando de tudo, o %ADF nao bate nem a pau.. coloquei o ADF2 e o ADF3 que estao na planilha do ferro.. mas ele alterna entre os dois de uma forma q nao entendi, baseada na importancia do coeficiente do tempo na regressao.. e mesmo assim parece q nao bateria.. apesar de que para 'CMIG3.SA','ECOR3.SA' bateu top



In [60]:
tickers = ['ALPA4','CSAN3'];
a = Get_Cubo(tickers,'2020-07-10-11-00',True)
b = Get_Cubo(tickers,'2020-07-10-11-15',True)
c = Get_Cubo(tickers,'2020-07-10-11-30',True)
d = Get_Cubo(tickers,'2020-07-10-11-45',True)
e = Get_Cubo(tickers,'2020-07-10-12-00',True)
f = Get_Cubo(tickers,'2020-07-10-12-15',True)

2019-07-10-17-00


,Periodo,Dickey_Fuller,ADF,ADF_0pct,TEMPO,ADF_99pct,Coef_Ang,QTD_Desvios,Fisher_min,Fisher_max,ADF_pvalue,pct_fin,meia_vida
0,100.00,-4.19,99%,95%,0%,99%,0.35,0.10,56%,78%,0.00,88%,5 Dias
1,120.00,-4.64,99%,99%,0%,99%,0.35,0.11,55%,75%,0.00,88%,5 Dias
2,140.00,-5.10,99%,99%,0%,99%,0.35,0.13,54%,73%,0.00,87%,5 Dias
3,160.00,-5.51,99%,99%,0%,99%,0.35,0.10,53%,72%,0.00,88%,5 Dias
4,180.00,-5.06,99%,99%,0%,99%,0.37,-0.42,54%,71%,0.00,107%,7 Dias
5,200.00,-4.88,99%,99%,0%,99%,0.39,-0.83,54%,71%,0.00,101%,9 Dias
6,220.00,-4.27,99%,95%,0%,99%,0.41,-1.06,53%,70%,0.00,97%,12 Dias
7,240.00,-4.14,99%,95%,0%,99%,0.42,-1.26,53%,69%,0.00,95%,15 Dias
8,250.00,-3.78,99%,95%,0%,99%,0.43,-1.36,53%,69%,0.00,94%,17 Dias


Desvio Medio do Coef. Angular: 8.61%
2019-07-10-17-00


,Periodo,Dickey_Fuller,ADF,ADF_0pct,TEMPO,ADF_99pct,Coef_Ang,QTD_Desvios,Fisher_min,Fisher_max,ADF_pvalue,pct_fin,meia_vida
0,100.00,-4.19,99%,95%,0%,99%,0.35,0.12,56%,78%,0.00,88%,5 Dias
1,120.00,-4.63,99%,99%,0%,99%,0.35,0.13,55%,75%,0.00,88%,5 Dias
2,140.00,-5.10,99%,99%,0%,99%,0.35,0.15,54%,73%,0.00,87%,5 Dias
3,160.00,-5.51,99%,99%,0%,99%,0.35,0.12,53%,72%,0.00,88%,5 Dias
4,180.00,-5.06,99%,99%,0%,99%,0.37,-0.40,54%,71%,0.00,107%,7 Dias
5,200.00,-4.88,99%,99%,0%,99%,0.39,-0.80,54%,71%,0.00,101%,9 Dias
6,220.00,-4.27,99%,95%,0%,99%,0.41,-1.04,53%,70%,0.00,97%,12 Dias
7,240.00,-4.14,99%,95%,0%,99%,0.42,-1.24,53%,69%,0.00,95%,15 Dias
8,250.00,-3.78,99%,95%,0%,99%,0.43,-1.34,53%,69%,0.00,94%,17 Dias


Desvio Medio do Coef. Angular: 8.61%
2019-07-10-17-00


...

Desvio Medio do Coef. Angular: 8.59%
2019-07-10-17-00


,Periodo,Dickey_Fuller,ADF,ADF_0pct,TEMPO,ADF_99pct,Coef_Ang,QTD_Desvios,Fisher_min,Fisher_max,ADF_pvalue,pct_fin,meia_vida
0,100.00,-4.19,99%,95%,0%,99%,0.35,0.11,56%,78%,0.00,88%,5 Dias
1,120.00,-4.64,99%,99%,0%,99%,0.35,0.12,55%,75%,0.00,87%,5 Dias
2,140.00,-5.10,99%,99%,0%,99%,0.35,0.14,54%,73%,0.00,87%,5 Dias
3,160.00,-5.51,99%,99%,0%,99%,0.35,0.11,53%,72%,0.00,88%,5 Dias
4,180.00,-5.06,99%,99%,0%,99%,0.37,-0.40,54%,71%,0.00,107%,7 Dias
5,200.00,-4.88,99%,99%,0%,99%,0.39,-0.79,54%,71%,0.00,101%,9 Dias
6,220.00,-4.27,99%,95%,0%,99%,0.41,-1.02,54%,70%,0.00,98%,12 Dias
7,240.00,-4.14,99%,95%,0%,99%,0.42,-1.22,53%,69%,0.00,95%,15 Dias
8,250.00,-3.78,99%,95%,0%,99%,0.43,-1.31,53%,69%,0.00,94%,17 Dias


Desvio Medio do Coef. Angular: 8.62%
2019-07-10-17-00


,Periodo,Dickey_Fuller,ADF,ADF_0pct,TEMPO,ADF_99pct,Coef_Ang,QTD_Desvios,Fisher_min,Fisher_max,ADF_pvalue,pct_fin,meia_vida
0,100.00,-4.19,99%,95%,0%,99%,0.35,0.07,56%,78%,0.00,88%,5 Dias
1,120.00,-4.64,99%,99%,0%,99%,0.35,0.09,55%,75%,0.00,88%,5 Dias
2,140.00,-5.11,99%,99%,0%,99%,0.35,0.11,54%,73%,0.00,87%,5 Dias
3,160.00,-5.51,99%,99%,0%,99%,0.35,0.07,53%,72%,0.00,88%,5 Dias
4,180.00,-5.06,99%,99%,0%,99%,0.37,-0.44,54%,71%,0.00,107%,7 Dias
5,200.00,-4.88,99%,99%,0%,99%,0.39,-0.83,54%,71%,0.00,101%,9 Dias
6,220.00,-4.27,99%,95%,0%,99%,0.41,-1.07,53%,70%,0.00,97%,12 Dias
7,240.00,-4.14,99%,95%,0%,99%,0.42,-1.25,53%,69%,0.00,95%,15 Dias
8,250.00,-3.78,99%,95%,0%,99%,0.43,-1.35,53%,69%,0.00,94%,17 Dias


Desvio Medio do Coef. Angular: 8.62%


In [ ]:
# Selecionar o periodo para mostrar o gráfico dos residuos
#for Periodo in [100,120,140,160,180,200,220,240,250]:
    
    #print (Periodo)
Periodo = 160
zscore = regressao_multivariada(par[tickers[1]].tail(Periodo), par[tickers[0]].tail(Periodo),Periodo)[2]
grafico_residuo(zscore)

coef_trade = cubo[cubo['Periodo']==160]['Coef_Ang']
dp_target = 1
dp_entrada = 2.4
time_trade = '2020-07-21-11-00'
print (time_trade[:-6])
h_life = int(cubo.loc[cubo['Periodo']==160, 'meia_vida'].item().split(" ")[0])

simula_trade(tickers, coef_trade, dp_entrada, dp_target, time_trade, h_life)



#print(zscore)
#@ipywidgets.interact(Periodo=(["Selecione o Período", 100,120,140,160,180,200,220,240,250]))
#def Grafico(Periodo):
#   if (Periodo != "Selecione o Período"):      

In [ ]:
['date', 'time', 'periodo', 'ratio', 'adf_stat', 'adf_sign', 'coef_Ang', 'desvio', 'pct_fin', 'meia_vida']
['2018-02-01', '10:05', 100, 2.4, 4.5, "95%", 0.39, 0.40, 10]

In [14]:
Cubo = {'date': ['2020-08-03-11:00','2020-08-03-11:00','2020-08-03-11:00'], 'periodo': [100,120,140],
        'ratio': [0.35 , 0.45, 0.40], 'adf_stat':[-3.2, -3.85, -4.75], 'adf_sign':[0.9, 0.95, 0.99], 
        'coef_Ang':[2.05, 1.95, 1.98], 'desvio':[2.2, 2, 2.1], 'pct_fin':[0.7, 0.65, 0.88], 'meia_vida':[10,9,8]}

In [15]:
Cubo

{'date': ['2020-08-03-11:00', '2020-08-03-11:00', '2020-08-03-11:00'],
 'periodo': [100, 120, 140],
 'ratio': [0.35, 0.45, 0.4],
 'adf_stat': [-3.2, -3.85, -4.75],
 'adf_sign': [0.9, 0.95, 0.99],
 'coef_Ang': [2.05, 1.95, 1.98],
 'desvio': [2.2, 2, 2.1],
 'pct_fin': [0.7, 0.65, 0.88],
 'meia_vida': [10, 9, 8]}

In [16]:
pd.DataFrame(Cubo)

,date,periodo,ratio,adf_stat,adf_sign,coef_Ang,desvio,pct_fin,meia_vida
0,2020-08-03-11:00,100,0.35,-3.20,0.90,2.05,2.20,0.70,10
1,2020-08-03-11:00,120,0.45,-3.85,0.95,1.95,2.00,0.65,9
2,2020-08-03-11:00,140,0.40,-4.75,0.99,1.98,2.10,0.88,8
